# Exploration de la base de données

In [1]:
#!conda install pandas #dans la console
from datetime import date, timedelta
import os

import pandas as pd

In [2]:
# Racine des fichiers quotidiens
BASE_URL = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{}.csv'

# Dates de diponibilité des fichiers
START_DATE = date(2020, 1, 22)
END_DATE = date(2020, 3, 20)

#Répertoire de sauvegarde des fichiers bruts
RAWFILES_DIR = '../data/raw/'
PROCESSED_DIR = '../data/processed/'

## Boucle de récupération des fichiers

In [3]:
delta = END_DATE - START_DATE       # as timedelta

for i in range(delta.days + 1):
    day = START_DATE + timedelta(days=i)
    day_label = day.strftime("%m-%d-%Y")
    
    virus_df = pd.read_csv(BASE_URL.format(day_label), sep=",", parse_dates=["Last Update"])
    virus_df.to_csv(os.path.join(RAWFILES_DIR, day_label + '.csv'), index=False)
    
    #print(day_label)

In [4]:
virus_df.dtypes

Province/State            object
Country/Region            object
Last Update       datetime64[ns]
Confirmed                  int64
Deaths                     int64
Recovered                  int64
Latitude                 float64
Longitude                float64
dtype: object

## Constitution de la table de référence lat/long

In [5]:
import glob

df_list = []

# Lecture des fichiers récupérés et sélection de ceux qui ont une lat / long
for file in glob.glob(os.path.join(RAWFILES_DIR, '*.csv')):
    virus_df = pd.read_csv(file, sep=',')
    if 'Latitude' in virus_df.columns and 'Longitude' in virus_df.columns:
        df_list.append(virus_df)

all_df = pd.concat(df_list)

# Table de référence pour les lat / long
(all_df[["Province/State", "Country/Region", "Latitude", "Longitude"]]
 .drop_duplicates(subset=["Province/State", "Country/Region"])
 .sort_values(by=["Country/Region", "Province/State"])
 .to_csv(os.path.join(PROCESSED_DIR, "lat_long_table.csv"), index=False)
)

In [6]:
(all_df[["Province/State", "Country/Region", "Latitude", "Longitude"]]
 .drop_duplicates()
 .shape
)

(523, 4)

In [7]:
(all_df[["Province/State", "Country/Region", "Latitude", "Longitude"]]
 .drop_duplicates()
 .drop_duplicates(subset=["Province/State", "Country/Region"])
 .shape
)
# 3 villes qui ont des latitudes-longitudes différentes

(426, 4)

In [8]:
(all_df[["Province/State", "Country/Region", "Latitude", "Longitude"]]
 .drop_duplicates()
 [(all_df[["Province/State", "Country/Region", "Latitude", "Longitude"]]
   .drop_duplicates()
   .duplicated(subset=["Province/State", "Country/Region"], keep=False))]
)
# Les 3 pays qui ont "bougé".

,Province/State,Country/Region,Latitude,Longitude
2,NaN,Italy,43.000000,12.000000
8,NaN,Iran,32.000000,53.000000
20,NaN,Japan,36.000000,138.000000
29,NaN,Germany,51.000000,9.000000
31,NaN,Singapore,1.283300,103.833300
35,NaN,Spain,40.000000,-4.000000
39,NaN,Bahrain,26.027500,50.550000
40,NaN,Kuwait,29.500000,47.750000
42,NaN,Thailand,15.000000,101.000000
45,NaN,Malaysia,2.500000,112.500000


## Construction d'une table unique

In [9]:
data_catalog = {
    'Last Update':["<M8[ns]"],
    "Confirmed":["float64", "int64"],
    "Deaths":["float64", "int64"],
    "Recovered":["float64", "int64"],
    "Latitude":["float64"],
    "Longitude":["float64"]
}

In [10]:
df_list = []

latlong_df = pd.read_csv(os.path.join(PROCESSED_DIR, "lat_long_table.csv"))

# Lecture des fichiers récupérés et sélection de ceux qui ont une lat / long
for file in glob.glob(os.path.join(RAWFILES_DIR, '*.csv')):
    virus_df = pd.read_csv(file, sep=',', parse_dates=["Last Update"])
    if not('Latitude' in virus_df.columns and 'Longitude' in virus_df.columns):
        virus_df = virus_df.merge(latlong_df, on=["Province/State", "Country/Region"], how='left')
    
    # Checker le type des variables dans l'importation de chaque fichier.
    for field, types in data_catalog.items():
        assert virus_df[field].dtypes in types, f"bad type for {field} in {file}"

    df_list.append(virus_df.assign(source=os.path.basename(file)))
 
all_df = pd.concat(df_list)

# Sauvegarde de la table totale
all_df.to_csv(os.path.join(PROCESSED_DIR, 'all_data.csv'), index=False)